## 0.1 Import dependencies

In [ ]:
# pylint: disable=W0104, W0212, E1120, E1101
import json
import logging
import os
import pathlib
import time

import ska_ser_logging
from bokeh.io import output_notebook

from ska_mid_jupyter_notebooks.dish.dish import TangoDishCluster
from ska_mid_jupyter_notebooks.helpers.path import project_root
from ska_mid_jupyter_notebooks.sut.configure import TangoSUTCluster, disable_qa
from ska_mid_jupyter_notebooks.test_equipment.configure import TangoTestEquipment

### 0.2 Setup Global Variables

In [ ]:
dev_mode = True
dishlmc_enabled = False
branch_name = "at-1981-cicd-updates"
test_equipment = TangoTestEquipment()
sut = TangoSUTCluster(branch_name, dev_mode)
dish_ids = ["0001", "0036"]
if dishlmc_enabled:
    dish_clusters = [
        TangoDishCluster(f"ska{d[1:]}", branch_name=branch_name, dev_mode=dev_mode)
        for d in dish_ids
    ]
else:
    dish_clusters = []
subarray_count = 1
subarray_id = 1

timestr = time.strftime("%Y%m%d-%H%M")
notebook_output_dir = pathlib.Path(
    project_root(), f"notebook-execution-data/configure_scan_for_commissioning/execution-{timestr}"
)
os.makedirs(notebook_output_dir)

### 0.3 Export System Configuration

In [ ]:
from ska_mid_jupyter_notebooks.cia import cia
from ska_mid_jupyter_notebooks.cluster.cluster import TangoCluster

cluster: TangoCluster
for cluster in [sut, test_equipment, *dish_clusters]:
    cia.export_chart_configuration(namespace=cluster.namespace, output_dir=notebook_output_dir)

## Step 1

### Use the noise source at nominal levels for input to at least one SPFRx.  

Expected Result:

### 1.1 Connect to Test Equipment 

In [ ]:
from ska_mid_jupyter_notebooks.test_equipment.state import get_equipment_model

test_equipment.smoke_test()
test_equipment.ignore("dserver")
test_equipment_state = get_equipment_model(test_equipment)
test_equipment.devices

### 1.2 Create Test Equipment Plot

In [ ]:
from ska_mid_jupyter_notebooks.test_equipment.rendering import get_test_equipment_monitor_plot

monitor_plot = get_test_equipment_monitor_plot()
test_equipment_state.subscribe_to_test_equipment_state(monitor_plot.handle_device_state_change)
output_notebook()
monitor_plot.show()
test_equipment_state.activate()

### 1.3 Display Test Equipment Device States 

In [ ]:
test_equipment_state.state["devices_states"]

### 1.4 Turn offline Test Equipment devices ON

In [ ]:
# set any offline devices to online
for dev in [
    test_equipment.__getattr__(test_equipment._replace(device))
    for device in test_equipment.devices
    if not ("dserver" in device)
]:
    if hasattr(dev, "adminMode"):
        if dev.adminMode != 0:
            dev.adminMode = 0

### 1.5 Configure Signal Generator and set noise
#### Correlated noise to be set to -132dbm/Hz
from skysim description: The Correlated Noise source simulated the SKY noise received by the Dishes. The amplitude of the Correlated noise at the 8 outputs are in phase and amplitude can be variable over a range of 30 dB in 0.25 dB steps. The nominal Noise power at the output is -130 dBm/Hz. The Correlated noise source can be switch ON or OFF remotely.

### Signal Generator does not need to be set for the configure scan test.  don't run

In [ ]:
frequency_to_set = 800e6

signal_generator = test_equipment.mid_itf_siggen_1
signal_generator.frequency = frequency_to_set
time.sleep(1)
assert (
    signal_generator.frequency == frequency_to_set
), f"Frequency required is {frequency_to_set} but got {signal_generator.frequency}"

## Step 2

### Use OSO scripting interface to TMC to run the script for assigning resources for a single sub-array, configuring and running a  scan.  

Expected Result:

### 2.1 Import dependencies

In [ ]:
from ska_oso_pdm.entities.common.target import (
    CrossScanParameters,
    FivePointParameters,
    RasterParameters,
    SinglePointParameters,
    StarRasterParameters,
)
from ska_oso_pdm.entities.sdp import BeamMapping
from ska_oso_scripting import oda_helper
from ska_oso_scripting.objects import SubArray, Telescope
from ska_tmc_cdm.messages.central_node.sdp import Channel
from ska_tmc_cdm.messages.subarray_node.configure.core import ReceiverBand
from tango import DeviceProxy

from ska_mid_jupyter_notebooks.obsconfig.config import ObservationSB
from ska_mid_jupyter_notebooks.obsconfig.target_spec import TargetSpec
from ska_mid_jupyter_notebooks.sut.rendering import TelescopeMononitorPlot
from ska_mid_jupyter_notebooks.sut.state import TelescopeDeviceModel, get_telescope_state

### 2.2 Initial Configuration
Setup logging, tango DB connection, etc.

In [ ]:
# we disable qa as it is not been properly verified
disable_qa()
enable_logging = True

if enable_logging:
    ska_ser_logging.configure_logging(logging.DEBUG)

sut.smoke_test()
for dish_cluster in dish_clusters:
    dish_cluster.smoke_test()
os.environ["TANGO_HOST"] = sut.tango_host()

### 2.3 Setup ODA

In [ ]:
os.environ["ODA_URI"] = (
    "http://ingress-nginx-controller-lb-default.ingress-nginx.svc.miditf.internal.skao.int/ska-db-oda/api/v1/"
)
eb_id = oda_helper.create_eb()
print(f"Execution Block ID: {eb_id}")

### 2.4 Setup monitoring & plot

In [ ]:
# setup monitoring
# use telescope state object for state monitoring
device_model = TelescopeDeviceModel(dish_ids, subarray_count)
telescope_state = get_telescope_state(device_model, sut)
# use monitor plot as a dashboard
telescope_monitor_plot = TelescopeMononitorPlot(plot_width=900, plot_height=200)
# set up events to monitor
telescope_state.subscribe_to_on_off(telescope_monitor_plot.observe_telescope_on_off)
telescope_state.subscribe_to_subarray_resource_state(
    telescope_monitor_plot.observe_subarray_resources_state
)
telescope_state.subscribe_to_subarray_configurational_state(
    telescope_monitor_plot.observe_subarray_configuration_state
)
telescope_state.subscribe_to_subarray_scanning_state(
    telescope_monitor_plot.observe_subarray_scanning_state
)
output_notebook()

### 2.5 Open the inline dashboard
Start the simple inline dashboard showing current state of the Telescope and resource assignment and configuration status.

In [ ]:
telescope_monitor_plot.show()
telescope_state.activate()
telescope_state.wait_til_ready(2)

### 2.6 Display Telescope State

In [ ]:
telescope_state.state

### 2.7 Initialise Telescope and Subarray
Create Subarray and Telescope instances.

In [ ]:
sub = SubArray(subarray_id)
tel = Telescope()

### 2.9 Reset the Telescope/Subarray
Set booleans to True to reset the system after a failed execution.

In [ ]:
do_reset_subarray = False
do_reset_csp_subarray = False
do_reset_dish = False

if do_reset_subarray:
    sub.abort()
    time.sleep(3)
    sub.restart()

if do_reset_csp_subarray:
    sut.reset_csp_subarray()

if do_reset_dish:
    for dish_cluster in dish_clusters:
        dish_cluster.reset_dish()

### 2.10 Turn CSP ON if it is OFF

In [ ]:
# Check if the telescope is currently OFF and update monitor appropriately
if telescope_monitor_plot.on_off_state == "OFF":  # e.g. purple
    sut.turn_csp_on()

#### Print out Dish Modes and Component States

In [ ]:
for dish_cluster in dish_clusters:
    dish_cluster.print_component_states()

### 2.11 Turn telescope ON

#### Load Dish-VCC Configuration in TMC

In [ ]:
sut.load_dish_cfg()

In [ ]:
# set to ON only if OFF

time.sleep(2)
if telescope_monitor_plot.on_off_state == "OFF":  # e.g. purple
    tel.on()
else:
    assert (
        telescope_monitor_plot.on_off_state == "ON"
    ), f"Cant continue with telescope in {telescope_monitor_plot.on_off_state}"

### 2.12 Observation Definition

#### 2.12.1 Define Resources to be used during Observation
Generate Processing Block and Execution Block IDs using SKUID.

In [ ]:
# External IDs (this would typically come from a database like ODA)
# this is simulated by means of instantiate a new Observation object comping from the helper modules
observation = ObservationSB()

#### 2.12.2 Create the high level observation specifications in terms of target specs

Note :- Users may currently modify the values by replacing the example values as given for each field within Target specification section.

In [ ]:
dish_ids = [d.dish_id for d in dish_clusters]
DEFAULT_TARGET_SPECS = {
    "flux calibrator": TargetSpec(
        target_sb_detail={
            "co_ordinate_type": "Equatorial",
            "ra": "19:24:51.05 degrees",
            "dec": "-29:14:30.12 degrees",
            "reference_frame": "ICRS",
            "unit": ("hourangle", "deg"),
            "pointing_pattern_type": {
                "single_pointing_parameters": SinglePointParameters(
                    offset_x_arcsec=0.0, offset_y_arcsec=0.0
                ),
                "raster_parameters": RasterParameters(
                    row_length_arcsec=0.0,
                    row_offset_arcsec=0.0,
                    n_rows=1,
                    pa=0.0,
                    unidirectional=False,
                ),
                "star_raster_parameters": StarRasterParameters(
                    row_length_arcsec=0.0,
                    n_rows=1,
                    row_offset_angle=0.0,
                    unidirectional=False,
                ),
                "five_point_parameters": FivePointParameters(offset_arcsec=0.0),
                "cross_scan_parameters": CrossScanParameters(offset_arcsec=0.0),
                "active_pointing_pattern_type": "single_pointing_parameters",
            },
        },
        scan_type="flux calibrator",
        band=ReceiverBand.BAND_2,
        channelisation="vis_channels9",
        polarisation="all",
        processing="test-receive-addresses",
        dish_ids=dish_ids,
        target=None,
    ),
    "M87": TargetSpec(
        target_sb_detail={
            "co_ordinate_type": "Equatorial",
            "ra": "19:24:51.05 degrees",
            "dec": "-29:14:30.12 degrees",
            "reference_frame": "ICRS",
            "unit": ("hourangle", "deg"),
            "pointing_pattern_type": {
                "single_pointing_parameters": SinglePointParameters(
                    offset_x_arcsec=0.0, offset_y_arcsec=0.0
                ),
                "raster_parameters": RasterParameters(
                    row_length_arcsec=0.0,
                    row_offset_arcsec=0.0,
                    n_rows=1,
                    pa=0.0,
                    unidirectional=False,
                ),
                "star_raster_parameters": StarRasterParameters(
                    row_length_arcsec=0.0,
                    n_rows=1,
                    row_offset_angle=0.0,
                    unidirectional=False,
                ),
                "five_point_parameters": FivePointParameters(offset_arcsec=0.0),
                "cross_scan_parameters": CrossScanParameters(offset_arcsec=0.0),
                "active_pointing_pattern_type": "single_pointing_parameters",
            },
        },
        scan_type="M87",
        band=ReceiverBand.BAND_2,
        channelisation="vis_channels10",
        polarisation="all",
        processing="test-receive-addresses",
        dish_ids=dish_ids,
        target=None,
    ),
}


channel_configuration = [
    Channel(
        spectral_window_id="fsp_1_channels",
        count=14880,
        start=0,
        stride=2,
        freq_min=0.35e9,
        freq_max=0.368e9,
        link_map=[[0, 0], [200, 1], [744, 2], [944, 3]],
    )
]

for key, value in DEFAULT_TARGET_SPECS.items():
    observation.add_channel_configuration(value.channelisation, channel_configuration)

observation.add_target_specs(DEFAULT_TARGET_SPECS)

for target_id, target in DEFAULT_TARGET_SPECS.items():
    observation.add_scan_type_configuration(
        config_name=target_id,
        beams={"vis0": BeamMapping(beam_id="vis0", field_id="M83")},
        derive_from=".default",
    )
scan_sequence = ["flux calibrator", "M87"]
observation.add_scan_sequence(scan_sequence)

### 2.13.1 Mid configuration schema input used by observing commands

[Configuration Schemas-OET→TMC(Mid)](https://confluence.skatelescope.org/display/SWSI/Configuration+Schemas#ConfigurationSchemas-OET%E2%86%92TMC(Mid))

In [ ]:
telescope_monitor_plot.show()

### 2.13.2 Create Scheduling Block Definition(SBD) Instance and save it into the ODA

In [ ]:
observation.eb_id = eb_id
pdm_allocation = observation.generate_pdm_object_for_sbd_save(DEFAULT_TARGET_SPECS)

print("\n\nSaving Scheduling Block Definition Instance in ODA\n\n")
sbd = oda_helper.save(pdm_allocation)
sbd_id = sbd.sbd_id
print(sbd_id)
pdm_allocation.sbd_id = sbd_id

In [ ]:
telescope_monitor_plot.show()

### 2.14 Assign Resources
Assign the requested resources to a Subarray

In [ ]:
assign_request = observation.generate_allocate_config_sb(pdm_allocation).as_object

In [ ]:
# Print assign resources request JSON out for debugging
from ska_oso_scripting.functions.devicecontrol.resource_control import get_request_json
from ska_tmc_cdm.messages.central_node.assign_resources import AssignResourcesRequest

print(f"AssignResourcesRequest: {assign_request}")
request_json = get_request_json(assign_request, AssignResourcesRequest, True)
print(request_json)
print(json.dumps(json.loads(request_json), indent=2))

In [ ]:
sub.assign_from_cdm(assign_request)
telescope_monitor_plot.show()

### 2.15 Configure and Run Multi Scan
Configure the telescope and run the scan on multiple targets

In [ ]:
from ska_tmc_cdm.messages.subarray_node.configure import ConfigureRequest

for scan_def_id in scan_sequence:
    configure_object = observation.generate_scan_config_sb(
        pdm_observation_request=pdm_allocation,
        scan_definition_id=scan_def_id,
        scan_duration=10.0,
    ).as_object
    cfg_json = get_request_json(configure_object, ConfigureRequest)
    print(f"ConfigureRequest={cfg_json}")
    sub.configure_from_cdm(configure_object, timeout=120)
    time.sleep(2)
    sub.scan(timeout=120)

In [ ]:
telescope_monitor_plot.show()

## 2.16 Post Observation teardown

#### 2.16.1 Clear scan configuration 

In [ ]:
sub.end()

In [ ]:
telescope_monitor_plot.show()

#### 2.16.2 Release Subarray resources

In [ ]:
sub.release()

In [ ]:
telescope_monitor_plot.show()

**Step 3:**

TMC configures Dishes (SPFRx, SPF simulator, DS simulator), CBF and SDP to the basic operational state for an imaging observational scan.  

Expected Result:  
__

**Step 4:**

TMC configures the system to run the scan.  

Expected Result:  
__

**Step 5:**

Show that each component enters and reports the correct observational state.  

Expected Result:  
__

**Step 6:**

Run the scan.  

Expected Result:  
__

**Step 7:**

Observe SDP signal displays for operation.  

Expected Result:  
__

**Step 8:**

Analyse recorded data to show that the scan runs and that the CBF produces channelised auto- and cross correlation visibilities ingested by SDP and stored in MS format (refer also to visibilities data products test). Use the Data Product Dashboard to download MS data.  

Expected Result:  
__